In [4]:
!pip uninstall -y transformers accelerate
!pip install "transformers==4.40.2" "accelerate==0.30.1" safetensors -U


Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ------------------------ --------------- 5.5/9.0 MB 28.0 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 29.4 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 9.0 MB/s  0:00:00

  Attempting uninstall: tokenizers

    Found existing installation: tokenizers 0.22.0

    Uninstalling tokenizers-0.22.0:

      Successfully uninstalled tokenizers-0.22.0

   ---------------------------------------- 0/3 [tokenizers]
   ---------------------------------------- 0/3 [tokenizers]
   --------------

  You can safely remove it manually.


In [3]:
import os, shutil
from pathlib import Path

loc = "facebook/nllb-200-distilled-600M"
if os.path.isdir(loc):
    print("로컬폴더 발견 → 삭제:", loc)
    shutil.rmtree(loc, ignore_errors=True)

cache = Path.home() / ".cache" / "huggingface" / "hub" / "models--facebook--nllb-200-distilled-600M"
print("캐시 삭제:", cache)
shutil.rmtree(cache, ignore_errors=True)

print("정리 완료")


캐시 삭제: C:\Users\user\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M
정리 완료


In [6]:
import shutil, os
from pathlib import Path

loc = "facebook/nllb-200-distilled-600M"
if os.path.isdir(loc):
    print("로컬폴더 발견 → 삭제:", loc)
    shutil.rmtree(loc, ignore_errors=True)

cache = Path.home() / ".cache" / "huggingface" / "hub" / "models--facebook--nllb-200-distilled-600M"
if cache.exists():
    print("캐시 발견 → 삭제:", cache)
    shutil.rmtree(cache, ignore_errors=True)

print("✅ 캐시/폴더 정리 완료")

캐시 발견 → 삭제: C:\Users\user\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M
✅ 캐시/폴더 정리 완료


In [8]:
# 진단+복구 셀 (Jupyter) — 복붙해서 실행하세요.
import os,sys,traceback,shutil,socket
from pathlib import Path
print("=== ENV & PATH ===")
print("python:", sys.executable)
try:
    import torch, transformers, huggingface_hub, safetensors
    print("torch:", getattr(torch,'__version__',None), "cuda:", getattr(torch.version,'cuda',None), "avail:", torch.cuda.is_available())
    print("transformers:", transformers.__version__)
    print("huggingface_hub:", huggingface_hub.__version__)
    print("safetensors:", safetensors.__version__)
except Exception as e:
    print("import-version check error:", e)
    traceback.print_exc()

print("\n=== CWD & FILES ===")
cwd = Path.cwd()
print("cwd:", cwd)
print("files in cwd (first 100):")
for i,f in enumerate(sorted(os.listdir(cwd))):
    if i<100:
        print(" ",f)
    else:
        break

model_id = "facebook/nllb-200-distilled-600M"
local_model_dir = cwd / model_id
cache_dir = Path.home()/".cache"/"huggingface"/"hub"/f"models--{model_id.replace('/','--')}"
print("\nlocal_model_dir exists:", local_model_dir.exists(), "->", local_model_dir)
print("cache_dir exists:", cache_dir.exists(), "->", cache_dir)

# 1) remove local folder if present
if local_model_dir.exists():
    try:
        print("Removing local folder:", local_model_dir)
        shutil.rmtree(local_model_dir)
        print("Removed local folder.")
    except Exception:
        print("Failed to remove local folder:")
        traceback.print_exc()

# 2) remove cache folder (safe)
if cache_dir.exists():
    try:
        print("Removing cache folder:", cache_dir)
        shutil.rmtree(cache_dir)
        print("Removed cache folder.")
    except Exception:
        print("Failed to remove cache folder:")
        traceback.print_exc()

# 3) network check to huggingface
print("\n=== NETWORK CHECK ===")
try:
    # quick DNS/socket test to huggingface.co
    host = "huggingface.co"
    print("resolving",host,":",socket.gethostbyname(host))
    import urllib.request, urllib.error
    try:
        u = urllib.request.urlopen("https://huggingface.co/models", timeout=15)
        print("HTTP status for huggingface.co/models:", u.status)
    except Exception as e:
        print("HTTP access check failed:", e)
except Exception as e:
    print("Network/DNS check failed:", e)
    traceback.print_exc()

# 4) try HfApi.model_info (reports permissions/problems)
print("\n=== HfApi.model_info check ===")
try:
    from huggingface_hub import HfApi
    api = HfApi()
    info = api.model_info(model_id)
    print("model_info ok. private?", getattr(info,'private',None))
except Exception as e:
    print("HfApi.model_info failed:")
    traceback.print_exc()

# 5) attempt to load using safe, with verbose try/except
print("\n=== ATTEMPT: from_pretrained(force_download=True, use_safetensors=True, low_cpu_mem_usage=True, device_map='auto') ===")
try:
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    tok = AutoTokenizer.from_pretrained(model_id, force_download=True)
    print("Tokenizer loaded OK.")
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id,
        use_safetensors=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        force_download=True,
        trust_remote_code=False
    )
    dev = None
    try:
        dev = model.get_input_embeddings().weight.device
    except Exception:
        try:
            dev = next(model.parameters()).device
        except Exception:
            dev = "unknown"
    print("Model loaded; param device:", dev)
    print("SUCCESS: model appears loaded. Try generate next.")
except Exception as e:
    print("LOAD FAILED. Full traceback below:")
    traceback.print_exc()

print("\n=== END DIAGNOSTIC ===")


=== ENV & PATH ===
python: c:\Users\user\miniconda3\envs\torch-gpu\python.exe
torch: 2.6.0.dev20241112+cu121 cuda: 12.1 avail: True
transformers: 4.40.2
huggingface_hub: 0.35.0
safetensors: 0.6.2

=== CWD & FILES ===
cwd: c:\Users\user\Downloads\mscd\archive
files in cwd (first 100):
  FPS_steam_reviews_20_games.csv
  data
  engtranslator.ipynb
  fast_out
  opus-mt-mul-en
  out
  weighted_score_above_08.csv
  weighted_score_above_08.csv.zip
  가설.md
  게임별_리뷰당시시간_긍정_부정평가.twb

local_model_dir exists: False -> c:\Users\user\Downloads\mscd\archive\facebook\nllb-200-distilled-600M
cache_dir exists: True -> C:\Users\user\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M
Removing cache folder: C:\Users\user\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M
Removed cache folder.

=== NETWORK CHECK ===
resolving huggingface.co : 3.168.178.31
HTTP status for huggingface.co/models: 200

=== HfApi.model_info check ===
model_info ok. private? False

=== ATTEMPT: from_pr

Traceback (most recent call last):
  File "c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\transformers\modeling_utils.py", line 3321, in from_pretrained
    resolved_archive_file, revision, is_sharded = auto_conversion(
  File "c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\transformers\safetensors_conversion.py", line 111, in auto_conversion
    raise e
  File "c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\transformers\safetensors_conversion.py", line 90, in auto_conversion
    sha = get_conversion_pr_reference(api, pretrained_model_name_or_path, **cached_file_kwargs)
  File "c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\transformers\safetensors_conversion.py", line 65, in get_conversion_pr_reference
    private = api.model_info(model_id).private
  File "c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\huggingface_hub\utils\_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
  File "c:\Users\user\miniconda3\envs

In [1]:
import sys, torch, transformers, huggingface_hub
print("python:", sys.executable)
print("torch:", torch.__version__, "cuda:", torch.version.cuda, "avail:", torch.cuda.is_available())
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)

c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


python: c:\Users\user\miniconda3\envs\torch-gpu\python.exe
torch: 2.6.0.dev20241112+cu121 cuda: 12.1 avail: True
transformers: 4.40.2
huggingface_hub: 0.35.0


In [1]:
import huggingface_hub
print(huggingface_hub.__version__)

0.24.6


In [3]:
import huggingface_hub; print(huggingface_hub.__version__)  # => 0.24.6 여야 함


0.24.6


In [7]:
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

repo = "facebook/nllb-200-distilled-600M"

# 1) bin 포함 모든 파일 다운로드
local_dir = snapshot_download(
    repo_id=repo,
    revision="main",
    local_dir_use_symlinks=False
)
print("downloaded to:", local_dir)

# 2) 로컬 경로에서 로드 (bin 허용)
tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    local_dir,
    use_safetensors=False,          # ✅ safetensors 강제 해제
    low_cpu_mem_usage=True,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=False,
    local_files_only=True,
)

dev = model.get_input_embeddings().weight.device
print("✅ model device:", dev)

# 3) 번역 테스트 (ko→en)
tok.src_lang = "kor_Hang"
x = tok("안녕하세요. 반갑습니다.", return_tensors="pt").to(dev)
bos_id = tok.convert_tokens_to_ids("eng_Latn")
y = model.generate(**x, max_new_tokens=50, forced_bos_token_id=bos_id)
print("=>", tok.batch_decode(y, skip_special_tokens=True)[0])


Fetching 9 files: 100%|██████████| 9/9 [00:30<00:00,  3.36s/it]


downloaded to: C:\Users\user\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M\snapshots\f8d333a098d19b4fd9a8b18f94170487ad3f821d
✅ model device: cuda:0
=> Hello. I'm very pleased to meet you.


In [8]:
# NLLB-200 distilled 600M, ko↔en 번역 유틸
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

REPO = "facebook/nllb-200-distilled-600M"

# 1) 로컬 스냅샷(모든 파일 포함: bin) — 최초 1회만 내려받음
local_dir = snapshot_download(REPO, revision="main", local_dir_use_symlinks=False)

# 2) 로컬에서 로드 (torch 2.6+ 이므로 .bin 허용)
tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    local_dir,
    use_safetensors=False,                    # <- 이 모델은 safetensors 미제공, bin 사용
    device_map="auto",                        # 자동 배치 (GPU/CPU)
    torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32),
    low_cpu_mem_usage=True,
    local_files_only=True,
)

# 간단 번역 함수
def translate(text: str, src="kor_Hang", tgt="eng_Latn", max_new_tokens=128):
    tok.src_lang = src
    dev = model.get_input_embeddings().weight.device
    x = tok(text, return_tensors="pt").to(dev)
    bos_id = tok.convert_tokens_to_ids(tgt)
    y = model.generate(**x, max_new_tokens=max_new_tokens, forced_bos_token_id=bos_id)
    return tok.batch_decode(y, skip_special_tokens=True)[0]

print("✅ device:", model.get_input_embeddings().weight.device)
print("ko→en:", translate("안녕하세요. 반갑습니다."))        # 예시
# print("en→ko:", translate("Nice to meet you.", "eng_Latn", "kor_Hang"))


Fetching 9 files: 100%|██████████| 9/9 [00:00<?, ?it/s]


✅ device: cuda:0
ko→en: Hello. I'm very pleased to meet you.


In [9]:
# 언어감지 설치 (순수 파이썬이라 바로 됨)
!pip install -q langdetect

import os, sys, math, gc, pandas as pd
from langdetect import detect
from tqdm import tqdm

# ⚠️ 경로/컬럼명만 맞춰줘
INPUT_PATH  = "weighted_score_above_08.csv"   # zip이면 "weighted_score_above_08.csv.zip"
OUTPUT_PATH = "weighted_score_above_08_translated.csv"
REVIEW_COL  = "review"                         # 너 파일의 텍스트 컬럼명으로 바꿔줘

# CSV/ZIP 자동 판별
if INPUT_PATH.endswith(".zip"):
    df = pd.read_csv(INPUT_PATH, compression="zip")
else:
    df = pd.read_csv(INPUT_PATH)

assert REVIEW_COL in df.columns, f"'{REVIEW_COL}' 컬럼을 찾지 못했어. 실제 컬럼명으로 REVIEW_COL을 바꿔줘."
print("rows:", len(df), "columns:", list(df.columns)[:12])


C:\Users\user\AppData\Local\Temp\ipykernel_28264\726360570.py:17: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_PATH)


rows: 498094 columns: ['recommendationid', 'appid', 'game', 'author_steamid', 'author_num_games_owned', 'author_num_reviews', 'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played', 'language', 'review']


In [10]:
import pandas as pd

INPUT_PATH  = "weighted_score_above_08.csv.zip"   # zip 파일이라면 그대로 둬도 됨
OUTPUT_PATH = "weighted_score_above_08_translated.csv"

# CSV/ZIP 자동 판별
if INPUT_PATH.endswith(".zip"):
    df = pd.read_csv(INPUT_PATH, compression="zip", low_memory=False)
else:
    df = pd.read_csv(INPUT_PATH, low_memory=False)

print("rows:", len(df))
print("columns:", df.columns.tolist()[:15])  # 앞부분만 확인


rows: 498094
columns: ['recommendationid', 'appid', 'game', 'author_steamid', 'author_num_games_owned', 'author_num_reviews', 'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up']


In [1]:
# --- Steam language → NLLB 코드 매핑 + 번역 파이프라인 ---

import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import torch

INPUT_PATH  = "weighted_score_above_08.csv.zip"   # 입력 파일
OUTPUT_PATH = "weighted_score_above_08_translated.csv"
TEXT_COL    = "review"
LANG_COL    = "language"

# 0) 데이터 로드
df = pd.read_csv(INPUT_PATH, compression="zip" if INPUT_PATH.endswith(".zip") else None, low_memory=False)
assert TEXT_COL in df.columns and LANG_COL in df.columns

# 1) Steam 언어명 → NLLB 언어코드 매핑
STEAM2NLLB = {
    # 기본
    "english":"eng_Latn", "korean":"kor_Hang", "japanese":"jpn_Jpan",
    "schinese":"zho_Hans", "tchinese":"zho_Hant",
    "french":"fra_Latn", "german":"deu_Latn",
    "spanish":"spa_Latn", "spanish - spain":"spa_Latn",
    "latam":"spa_Latn", "spanish - latin america":"spa_Latn",
    "portuguese":"por_Latn", "brazilian":"por_Latn",
    "portuguese - brazil":"por_Latn", "portuguese - portugal":"por_Latn",
    "italian":"ita_Latn", "russian":"rus_Cyrl", "turkish":"tur_Latn",
    "arabic":"arb_Arab", "thai":"tha_Thai", "vietnamese":"vie_Latn", "indonesian":"ind_Latn",
    "polish":"pol_Latn", "dutch":"nld_Latn", "swedish":"swe_Latn", "finnish":"fin_Latn",
    "norwegian":"nob_Latn", "danish":"dan_Latn",
    "czech":"ces_Latn", "hungarian":"hun_Latn", "romanian":"ron_Latn",
    "greek":"ell_Grek", "ukrainian":"ukr_Cyrl",
    "bulgarian":"bul_Cyrl", "hebrew":"heb_Hebr",
    "croatian":"hrv_Latn", "serbian":"srp_Cyrl", "slovak":"slk_Latn",
    "malay":"zsm_Latn", "filipino":"tgl_Latn", "tagalog":"tgl_Latn",
}

def steam_to_nllb(s):
    if not isinstance(s, str): return None
    key = s.strip().lower()
    return STEAM2NLLB.get(key)

df["_nllb_src"] = df[LANG_COL].map(steam_to_nllb)

# 2) NLLB 모델/토크나이저 준비(이미 메모리에 있으면 재사용)
try:
    tok, model
    DEV = model.get_input_embeddings().weight.device
except NameError:
    from huggingface_hub import snapshot_download
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    REPO = "facebook/nllb-200-distilled-600M"
    local_dir = snapshot_download(REPO, revision="main", local_dir_use_symlinks=False)
    tok = AutoTokenizer.from_pretrained(local_dir, use_fast=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        local_dir,
        use_safetensors=False,        # 이 모델은 bin만 제공
        device_map="auto",
        torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32),
        low_cpu_mem_usage=True,
        local_files_only=True,
    )
    DEV = model.get_input_embeddings().weight.device
print("✅ device:", DEV)

def translate_batch(texts, src_nllb, tgt_nllb="eng_Latn", max_new_tokens=256):
    tok.src_lang = src_nllb
    x = tok(texts, return_tensors="pt", padding=True, truncation=True).to(DEV)
    bos_id = tok.convert_tokens_to_ids(tgt_nllb)
    with torch.no_grad():
        y = model.generate(**x, max_new_tokens=max_new_tokens, forced_bos_token_id=bos_id)
    return tok.batch_decode(y, skip_special_tokens=True)

# 3) 한국어/영어는 패스, 나머지만 영어로 번역
PASS = {"eng_Latn", "kor_Hang"}
df["translated_en"] = df[TEXT_COL].astype(str)

to_idx = [i for i,(txt,src) in enumerate(zip(df[TEXT_COL].astype(str), df["_nllb_src"]))
          if (txt.strip()!="") and (src is not None) and (src not in PASS)]

# 언어별로 묶어서 배치 번역
BATCH = 32
bucket = defaultdict(list)
for i in to_idx:
    bucket[df.loc[i,"_nllb_src"]].append(i)

for src, idxs in bucket.items():
    print(f"➡ {src} → eng_Latn : {len(idxs)} rows")
    for s in tqdm(range(0, len(idxs), BATCH), desc=f"{src}"):
        sub = idxs[s:s+BATCH]
        texts = df.loc[sub, TEXT_COL].astype(str).tolist()
        try:
            outs = translate_batch(texts, src)
        except Exception:
            # 실패 시 개별 처리(최소 손실)
            outs=[]
            for t in texts:
                try: outs.extend(translate_batch([t], src))
                except: outs.append(t)
        df.loc[sub, "translated_en"] = outs

# 4) 저장
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print("✅ saved ->", OUTPUT_PATH)


c:\Users\user\miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 9 files: 100%|██████████| 9/9 [00:00<?, ?it/s]


✅ device: cuda:0
➡ zho_Hans → eng_Latn : 160111 rows


zho_Hans: 100%|██████████| 5004/5004 [6:58:55<00:00,  5.02s/it]  


➡ rus_Cyrl → eng_Latn : 50557 rows


rus_Cyrl: 100%|██████████| 1580/1580 [2:15:49<00:00,  5.16s/it] 


...

➡ nld_Latn → eng_Latn : 18 rows


nld_Latn: 100%|██████████| 1/1 [00:04<00:00,  4.02s/it]


➡ dan_Latn → eng_Latn : 9 rows


dan_Latn: 100%|██████████| 1/1 [00:03<00:00,  3.91s/it]


✅ saved -> weighted_score_above_08_translated.csv


In [2]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 40.1 MB/s  0:00:00

   ---------------------------------------- 0/3 [joblib]
   ------------- -------------------------- 1/3 [click]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   ---------------------------------------- 3/3 [nltk]



In [4]:
import pandas as pd
from collections import Counter
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# 데이터 로드
df = pd.read_csv("weighted_score_above_08_translated.zip", compression="zip", low_memory=False)

# 영어 리뷰 텍스트
reviews = df["translated_en"].astype(str)

# 긍/부정 라벨 (여기선 voted_up 컬럼이 있다고 가정)
pos_reviews = df.loc[df["voted_up"]==True, "translated_en"].dropna().tolist()
neg_reviews = df.loc[df["voted_up"]==False, "translated_en"].dropna().tolist()

# 토큰화 + stopword 제거 함수
stop = set(stopwords.words("english"))
def tokenize(text):
    text = re.sub(r"[^a-z ]"," ", text.lower())
    return [w for w in text.split() if w not in stop and len(w)>2]

# 긍정/부정별 단어 카운트
pos_words = Counter()
for r in pos_reviews:
    pos_words.update(tokenize(r))

neg_words = Counter()
for r in neg_reviews:
    neg_words.update(tokenize(r))

# 상위 30개 키워드 뽑기
print("✅ 긍정 키워드 Top 30")
print(pos_words.most_common(30))
print("\n✅ 부정 키워드 Top 30")
print(neg_words.most_common(30))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ 긍정 키워드 Top 30
[('game', 836556), ('like', 161863), ('get', 145656), ('play', 142644), ('good', 137488), ('one', 132706), ('time', 119048), ('first', 104507), ('need', 97056), ('games', 95532), ('story', 85389), ('gameplay', 83961), ('also', 83723), ('really', 83642), ('going', 81451), ('new', 76886), ('want', 74233), ('even', 74049), ('people', 68408), ('make', 66346), ('much', 65666), ('playing', 65366), ('chinese', 64995), ('see', 62029), ('lot', 61954), ('would', 60443), ('well', 59097), ('fun', 58145), ('world', 57527), ('still', 57097)]

✅ 부정 키워드 Top 30
[('game', 255186), ('like', 49599), ('get', 47810), ('one', 37796), ('play', 37147), ('even', 36011), ('time', 35306), ('good', 27956), ('really', 27625), ('new', 25542), ('games', 23691), ('going', 23515), ('would', 23449), ('first', 23356), ('make', 22453), ('want', 21789), ('much', 20874), ('people', 20252), ('also', 20118), ('still', 19496), ('way', 18380), ('players', 18137), ('money', 18004), ('playing', 17257), ('buy', 172

In [3]:
import pandas as pd
import csv
from pathlib import Path

# 입력 파일들
SRC = "weighted_score_above_08_translated.zip"   # 너의 원본 (zip 또는 csv)
PATCH = "weighted_score_above_08_translated_patch_fixed_rows.csv"
OUT = "weighted_score_above_08_translated_patched.zip"  # 결과를 zip으로 저장

# 패치 로드
p = pd.read_csv(PATCH, dtype=str, low_memory=False)
has_id = "recommendationid" in p.columns

# 1차 매핑: recommendationid -> fixed
id_map = {}
if has_id:
    id_map = dict(p[["recommendationid","translated_en_fixed"]].dropna().values)

# 2차 매핑(백업): (language, review) -> fixed  (id가 없거나 일부 누락 대비)
key_map = {}
if "language" in p.columns and "review" in p.columns:
    key_map = dict(p[["language","review","translated_en_fixed"]]
                   .dropna().assign(language=lambda x: x["language"].str.strip().str.lower())
                   .set_index(["language","review"])["translated_en_fixed"].to_dict())

def apply_patch_chunk(df):
    # translated_en만 갱신
    if "translated_en" not in df.columns:
        return df

    # id 기준 교체
    if has_id and "recommendationid" in df.columns:
        m = df["recommendationid"].astype(str).map(id_map)
        df["translated_en"] = m.combine_first(df["translated_en"])

    # (language, review) 기준 보조 교체
    if key_map and {"language","review"}.issubset(df.columns):
        lang = df["language"].astype(str).str.strip().str.lower()
        key = list(zip(lang, df["review"].astype(str)))
        m2 = pd.Series([key_map.get(k) for k in key], index=df.index, dtype="object")
        df["translated_en"] = m2.combine_first(df["translated_en"])

    return df

# 청크 처리 + 저장
reader = pd.read_csv(
    SRC,
    compression="zip" if str(SRC).endswith(".zip") else None,
    low_memory=False,
    chunksize=120_000,
    dtype=str,
    encoding="utf-8"
)

first = True
for chunk in reader:
    chunk = apply_patch_chunk(chunk)
    chunk.to_csv(
         "tmp_patched.csv",
         mode="w" if first else "a",
         index=False,
         header=first,
         encoding="utf-8-sig",
          quoting=csv.QUOTE_ALL,
          escapechar="\\",
          lineterminator="\n",   # ✅ 여기! line_terminator(X) → lineterminator(O)
        )
    first = False

# zip으로 감싸기 (엑셀 열 일 있으면 csv 그대로 써도 됨)
import zipfile, os
with zipfile.ZipFile(OUT, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("tmp_patched.csv", arcname="weighted_score_above_08_translated_patched.csv")
os.remove("tmp_patched.csv")

print("done ->", OUT)


done -> weighted_score_above_08_translated_patched.zip


In [4]:
import re, pandas as pd
moji = re.compile(r"(Ã.|Â.|â..|ï..|ðŸ|鈥|�)")
df = pd.read_csv("weighted_score_above_08_translated_patched.zip", compression="zip", low_memory=False, dtype=str, encoding="utf-8")
rate = df["translated_en"].astype(str).str.contains(moji).mean()
print("mojibake rate:", round(rate*100, 5), "%")


C:\Users\user\AppData\Local\Temp\ipykernel_22944\2000052898.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rate = df["translated_en"].astype(str).str.contains(moji).mean()


mojibake rate: 0.0 %


In [8]:
import pandas as pd
import re

# 원본 CSV 로드 (경로 수정)
df = pd.read_csv("weighted_score_above_08_translated_patched.csv")

# 정리 함수 정의
def clean_text(text):
    if pd.isna(text):
        return text
    # 1) '�' 같은 깨진 문자 제거
    text = text.replace("�", "")
    # 2) 라틴계 인코딩 잔재 (Ã, Â 등) 제거
    text = re.sub(r"[ÃÂÏÐ]", "", text)
    # 3) 필요하다면 제어문자 제거
    text = re.sub(r"[\x00-\x1F]", "", text)
    return text

# translated_en 컬럼 정리
df["translated_en"] = df["translated_en"].astype(str).apply(clean_text)

# 새 파일 저장
df.to_csv("weighted_score_above_08_translated_clean.csv", index=False)


C:\Users\user\AppData\Local\Temp\ipykernel_22944\25629504.py:5: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("weighted_score_above_08_translated_patched.csv")


In [1]:
import pandas as pd
import re

df = pd.read_csv("weighted_score_above_08_translated_patched.csv", low_memory=False)

# 정규식 패턴
BROKEN_RE = re.compile(r"[\ufffd\x00-\x08\x0B\x0C\x0E-\x1F\x7F]")
URL_RE = re.compile(r"(https?://\S+|www\.\S+)")
EMAIL_RE = re.compile(r"\b[\w.\-+]+@[\w.\-]+\.\w+\b")
MENTION_RE = re.compile(r"[@#][\w\-_.]+")
REPEAT_CHAR_RE = re.compile(r"(.)\1{4,}")      
NONWORD_STREAK_RE = re.compile(r"[^\w\s\u1100-\u11FF\u3130-\u318F\uAC00-\uD7A3]{6,}")
MULTI_SYMBOL_LINE_RE = re.compile(r"^[^\w\u1100-\u11FF\u3130-\u318F\uAC00-\uD7A3]{8,}$")

def clean_text(text: str) -> str:
    if pd.isna(text):
        return ""
    s = str(text)

    # 깨진 문자 / 제어문자
    s = BROKEN_RE.sub("", s)

    # URL / 이메일 / 멘션 제거
    s = URL_RE.sub("", s)
    s = EMAIL_RE.sub("", s)
    s = MENTION_RE.sub("", s)

    # 공백 정리
    s = re.sub(r"[ \t]{2,}", " ", s).strip()

    # 잡음 판정
    lines = [ln.strip() for ln in s.splitlines() if ln.strip()]
    noisy = 0
    for ln in lines:
        if MULTI_SYMBOL_LINE_RE.match(ln):
            noisy += 1
        elif NONWORD_STREAK_RE.search(ln) or REPEAT_CHAR_RE.search(ln):
            noisy += 1
        elif len(ln) >= 40:
            letters = sum(ch.isalnum() or ('\uAC00' <= ch <= '\uD7A3') for ch in ln)
            if letters / max(len(ln), 1) < 0.25:
                noisy += 1
    if noisy >= max(1, int(0.4 * len(lines))):
        return ""  # 노이즈 블록은 빈칸 처리

    return s

# translated_en 정리
df["translated_en"] = df["translated_en"].apply(clean_text)

# ===== 행 삭제: translated_en 이 빈칸/NaN 인 행 drop =====
df = df[df["translated_en"].notna() & (df["translated_en"].str.strip() != "")]

# 저장
df.to_csv("weighted_score_above_08_translated_clean.csv", index=False, encoding="utf-8-sig")

print("최종 행 수:", len(df))


최종 행 수: 478708


In [3]:
import os
import re
import pandas as pd
import torch
from pathlib import Path
from typing import Optional, List, Dict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ========================
# 경로/설정 (경로만 수정)
# ========================
SRC = Path(r"C:\YOUR\PATH\weighted_score_above_08_translated_clean.csv")   # 입력 CSV
DST = Path(r"C:\YOUR\PATH\weighted_score_above_08_translated_clean_final.csv")  # 결과 CSV
REMOVED = Path(r"C:\YOUR\PATH\removed_rows_after_retranslate.csv")              # 제거 목록 CSV

CHUNKSIZE = 100_000
BATCH_SIZE = 8
MAX_LEN = 512
ENGINE = "python"      # 문제 없으면 기본값으로 바꿔도 됨
ENCODING = "utf-8-sig" # 저장시 Excel 호환

# ========================
# “영어만 허용” 규칙 (ASCII만 허용)
# ========================
NON_ASCII = re.compile(r"[^\x09\x0A\x0D\x20-\x7E]")
BROKEN_RE = re.compile(r"[\ufffd\x00-\x08\x0B\x0C\x0E-\x1F\x7F]")  # � 및 제어문자

def is_english_only(s: str) -> bool:
    return (not s) or (NON_ASCII.search(s) is None)

def clean_basic(s: str) -> str:
    s = BROKEN_RE.sub("", s)
    s = re.sub(r"[ \t]{2,}", " ", s).strip()
    return s

# ========================
# 스크립트 기반 소스언어 추정
# ========================
SCRIPT_MAP = [
    (re.compile(r"[\u3040-\u309F\u30A0-\u30FF\u31F0-\u31FF]"), "jpn_Jpan"),
    (re.compile(r"[\u0400-\u04FF]"), "rus_Cyrl"),
    (re.compile(r"[\u0600-\u06FF]"), "arb_Arab"),
    (re.compile(r"[\u0900-\u097F]"), "hin_Deva"),
    (re.compile(r"[\u0370-\u03FF]"), "ell_Grek"),
    (re.compile(r"[\u0590-\u05FF]"), "heb_Hebr"),
    (re.compile(r"[\u0E00-\u0E7F]"), "tha_Thai"),
    (re.compile(r"[\u3400-\u9FFF\uF900-\uFAFF]"), "zho_Hans"),
    (re.compile(r"[\u1100-\u11FF\u3130-\u318F\uAC00-\uD7A3]"), "kor_Hang"),
]
def guess_src_lang_one(s: str) -> Optional[str]:
    for pat, code in SCRIPT_MAP:
        if pat.search(s):
            return code
    return None

# ========================
# NLLB 모델 로딩 (예외 안전)
# ========================
MODEL_NAME = "facebook/nllb-200-distilled-600M"
print("Loading model/tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Device: {device}")

# ─ Fix: env별 tokenizer 차이 보정
ENG_ID = None
if hasattr(tokenizer, "lang_code_to_id") and isinstance(tokenizer.lang_code_to_id, dict):
    ENG_ID = tokenizer.lang_code_to_id.get("eng_Latn")
if ENG_ID is None:
    try:
        ENG_ID = tokenizer.convert_tokens_to_ids("eng_Latn")
    except Exception:
        pass
if ENG_ID is None:
    raise RuntimeError("영어 토큰 ID(eng_Latn)를 찾을 수 없습니다. transformers 최신 버전으로 업데이트 해보세요.")

def set_src_lang_safe(lang_code: str):
    """토크나이저 구현마다 src_lang 유무가 달라서 안전 세팅"""
    if hasattr(tokenizer, "src_lang"):
        try:
            tokenizer.src_lang = lang_code
        except Exception:
            pass  # 없는 구현도 있음

# ========================
# 번역 유틸 (그룹/배치)
# ========================
def batch_translate_group(texts: List[str], src_lang: Optional[str]) -> List[str]:
    outs: List[str] = []
    effective_src = src_lang or "eng_Latn"  # 추정 실패 시 영어로 간주
    set_src_lang_safe(effective_src)

    start = 0
    while start < len(texts):
        batch = texts[start:start+BATCH_SIZE]
        start += BATCH_SIZE
        enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN).to(device)
        try:
            gen = model.generate(
                **enc,
                forced_bos_token_id=ENG_ID,
                max_length=MAX_LEN
            )
        except torch.cuda.OutOfMemoryError:
            # ─ Fix: OOM 시 배치 줄여 재시도
            torch.cuda.empty_cache()
            sub_outs = []
            for t in batch:
                enc1 = tokenizer([t], return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN).to(device)
                gen1 = model.generate(**enc1, forced_bos_token_id=ENG_ID, max_length=MAX_LEN)
                sub_outs.append(tokenizer.decode(gen1[0], skip_special_tokens=True))
            outs.extend([clean_basic(x) for x in sub_outs])
            continue

        out = tokenizer.batch_decode(gen, skip_special_tokens=True)
        outs.extend([clean_basic(x) for x in out])
    return outs

def retranslate_to_english(texts: List[str]) -> List[str]:
    groups: Dict[str, List[int]] = {}
    for i, t in enumerate(texts):
        lang = guess_src_lang_one(t) or "eng_Latn"
        groups.setdefault(lang, []).append(i)

    results = [""] * len(texts)
    for lang, idxs in groups.items():
        batch = [texts[i] for i in idxs]
        try:
            translated = batch_translate_group(batch, src_lang=lang)
        except Exception:
            translated = batch  # 번역 실패 시 원문 유지 → 이후 필터에서 제거될 수 있음
        for k, i in enumerate(idxs):
            results[i] = translated[k]
    return results

# ========================
# 메인 루프 (재번역→검증→삭제)
#  - translated_en에 비ASCII(=비영어) 1자라도 있으면 재번역
#  - 재번역 후에도 비ASCII or 빈문자열이면 행 삭제
# ========================
first_out, first_rm = True, True
total = bad_before = retranslated = removed = 0

# ─ Fix: 일부 pandas 버전은 TextFileReader가 context manager 미지원
reader = pd.read_csv(SRC, chunksize=CHUNKSIZE, low_memory=False, engine=ENGINE)
try:
    for chunk_idx, chunk in enumerate(reader, 1):
        if "translated_en" not in chunk.columns:
            raise KeyError("translated_en 컬럼이 없습니다.")

        texts = chunk["translated_en"].astype("string")
        total += len(texts)

        needs_fix_mask = texts.fillna("").apply(lambda x: not is_english_only(str(x)))
        need_idxs = texts.index[needs_fix_mask]
        bad_before += int(needs_fix_mask.sum())

        if len(need_idxs) > 0:
            to_fix = texts.loc[need_idxs].fillna("").astype(str).tolist()
            fixed = retranslate_to_english(to_fix)
            retranslated += len(fixed)
            chunk.loc[need_idxs, "translated_en"] = fixed

        final_texts = chunk["translated_en"].fillna("").astype(str)
        drop_mask = final_texts.apply(lambda x: (x.strip() == "") or (not is_english_only(x)))
        removed += int(drop_mask.sum())

        kept = chunk.loc[~drop_mask].copy()
        kept.to_csv(
            DST,
            mode="w" if first_out else "a",
            header=first_out,
            index=False,
            encoding=ENCODING
        )
        first_out = False

        if drop_mask.any():
            chunk.loc[drop_mask].to_csv(
                REMOVED,
                mode="w" if first_rm else "a",
                header=first_rm,
                index=False,
                encoding=ENCODING
            )
            first_rm = False

        print(f"[Chunk {chunk_idx}] in={len(chunk):6d}  need_fix={int(needs_fix_mask.sum()):5d}  "
              f"retranslated={len(need_idxs):5d}  removed_now={int(drop_mask.sum()):5d}")
finally:
    try:
        reader.close()
    except Exception:
        pass

print("\n=== Summary ===")
print(f"Total rows processed : {total}")
print(f"Rows needed fix      : {bad_before}")
print(f"Rows retranslated    : {retranslated}")
print(f"Rows removed (final) : {removed}")
print(f"Output file          : {DST}")
print(f"Removed log          : {REMOVED}")


Loading model/tokenizer...
Device: cuda


ValueError: The 'low_memory' option is not supported with the 'python' engine